# Column Densities with AbsComponent

In [ ]:
%matplotlib inline

In [2]:
# imports
try:
    import seaborn as sns; sns.set_style("white")
except:
    pass

from scipy import integrate

from linetools.isgm import abscomponent as lt_abscomp
from linetools.spectralline import AbsLine
from linetools.spectra import io as lsio
#
import imp
lt_path = imp.find_module('linetools')[1]

## Read Spec

In [3]:
xspec = lsio.readspec(lt_path+'/spectra/tests/files/UM184_nF.fits')

## Generate a few AbsLines

In [4]:
SiIItrans = ['SiII 1260', 'SiII 1304', 'SiII 1526', 'SiII 1808']

In [5]:
abslines = []
for trans in SiIItrans:
    iline = AbsLine(trans)
    iline.attrib['z'] = 2.92939
    iline.analy['vlim'] = [-250.,80.]*u.km/u.s
    iline.analy['spec'] = xspec
    abslines.append(iline)
#
abslines

linetools.lists.parse: Reading linelist --- 
   /Users/xavier/local/Python/linetools/linetools/data/lines/morton03_table2.fits.gz
linetools.lists.parse: Reading linelist --- 
   /Users/xavier/local/Python/linetools/linetools/data/lines/morton00_table2.fits.gz
linetools.lists.parse: Reading linelist --- 
   /Users/xavier/local/Python/linetools/linetools/data/lines/verner94_tab6.fits


linetools.lists.parse: Reading linelist --- 
   /Users/xavier/local/Python/linetools/linetools/data/lines/EUV_lines.ascii
read_sets: Using set file -- 
  /Users/xavier/local/Python/linetools/linetools/lists/sets/llist_v0.3.ascii


[[AbsLine: SiII 1260, wrest=1260.4221 Angstrom],
 [AbsLine: SiII 1304, wrest=1304.3702 Angstrom],
 [AbsLine: SiII 1526, wrest=1526.7070 Angstrom],
 [AbsLine: SiII 1808, wrest=1808.0129 Angstrom]]

## Generate the Component

In [6]:
reload(lt_abscomp)
abscomp = lt_abscomp.AbsComponent.from_abslines(abslines)

In [ ]:
abscomp.stack_plot()

## Measure Column Densities

In [ ]:
abscomp.measure_colm(redo_indiv=True)

In [ ]:
abscomp

In [ ]:
for iline in abscomp._abslines:
    print(iline.wrest, iline.attrib['flagN'], iline.attrib['logN'], iline.attrib['sig_logN'])

## COG

### $F(\tau_0)$

Definition
$F(\tau_0) = \int_0^\infty dx \, [1- \rm e^{-\tau_0 \rm e^{-x^2}}]$

In [ ]:
def ftau_intgrnd(x,tau0=0.1):
    return 1 - np.exp(-tau0 * np.exp(-x**2))

In [ ]:
neval = 10000
lgt = np.linspace(-3, 9, neval)
all_tau0 = 10.**lgt
Ftau = np.zeros(neval)
for jj,tau0 in enumerate(all_tau0):
    Ftau[jj], ferr = integrate.quad(ftau_intgrnd, 0, np.inf, args=(tau0,))

In [ ]:
# Damped limit (not accurate enough)
damp_lgt = np.linspace(6, 10, 100)
damp_tau0 = 10.**damp_lgt
damp_Ftau = np.sqrt(np.log(damp_tau0))

In [ ]:
xdb.xplot(lgt,Ftau, xtwo=damp_lgt, ytwo=1.015*damp_Ftau)

### Measure

In [54]:
reload(lt_abscomp)
abscomp = lt_abscomp.AbsComponent.from_abslines(abslines)
abscomp.cog(redo_indiv=True)